In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')
df.head()

In [ ]:
def weird_division(n, d):
    return n / d if d else 0


df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)), x['measurable_impressions'])*1000 , axis=1)
df.drop(columns=['total_revenue'], inplace=True)
df = df[df['CPM'] >= 0]
df['date'] = pd.to_datetime(df['date'])


In [ ]:
outlier = df[(df['date'] >= '2019-06-22')]['CPM'].quantile(0.95)

train = df[(df['date'] < '2019-06-22') & (df['CPM'] < outlier) ]
test = df[(df['date'] >= '2019-06-22') & (df['CPM'] < outlier) ]


In [ ]:
X_train = train.drop(columns=['date', 'CPM'], axis = 0)
y_train = train['CPM']

X_test = test.drop(columns=['date','CPM'], axis = 0)
y_test = test['CPM']

In [ ]:
xgb = XGBRegressor(random_state = 42)

xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)
MSE(y_test, y_pred)